# Evaluate the performance of the different models by using a sliding window approach

Steps:
1. Harmonize the predictions to have the same format
2. Extract the frequencies using a sliding window approach
3. Evaluate the performance of the models
4. Visualize the results

## Step 1: Harmonize the predictions

In [ ]:
import numpy as np
import pandas as pd
import respiration.utils as utils

signals_dir = utils.dir_path('outputs', 'signals')

In [ ]:
raft_file = utils.join_paths(signals_dir, 'raft_predictions.csv')
raft_predictions = pd.read_csv(raft_file)
raft_predictions['signal'] = raft_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
raft_predictions = raft_predictions[raft_predictions['roi'] == 'chest']

# Only keep the columns that are needed
raft_predictions = raft_predictions[['subject', 'setting', 'model', 'signal']]

raft_predictions.head()

In [ ]:
flownet_file = utils.join_paths(signals_dir, 'flownet_predictions.csv')
flownet_predictions = pd.read_csv(flownet_file)
flownet_predictions['signal'] = flownet_predictions['signal_v'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
flownet_predictions = flownet_predictions[flownet_predictions['roi'] == 'chest']

# Only keep the columns that are needed
flownet_predictions = flownet_predictions[['subject', 'setting', 'model', 'signal']]

flownet_predictions.head()

In [ ]:
pretrained_file = utils.join_paths(signals_dir, 'pretrained_predictions.csv')
pretrained_predictions = pd.read_csv(pretrained_file)
pretrained_predictions['signal'] = pretrained_predictions['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
pretrained_predictions = pretrained_predictions[['subject', 'setting', 'model', 'signal']]

pretrained_predictions.head()

In [ ]:
unsupervised_file = utils.join_paths(signals_dir, 'unsupervised_predictions.csv')
unsupervised_predictions = pd.read_csv(unsupervised_file)
unsupervised_predictions['signal'] = unsupervised_predictions['signal'].apply(eval).apply(np.array)

# Only keep the chest roi predictions
unsupervised_predictions = unsupervised_predictions[unsupervised_predictions['roi'] == 'chest']

# Rename column method to model
unsupervised_predictions.rename(columns={'method': 'model'}, inplace=True)

# Only keep the columns that are needed
unsupervised_predictions = unsupervised_predictions[['subject', 'setting', 'model', 'signal']]

unsupervised_predictions.head()

In [ ]:
fine_tuned_path = utils.join_paths(signals_dir, 'fine_tuned_predictions.csv')

fine_tuned_prediction = pd.read_csv(fine_tuned_path)
fine_tuned_prediction['signal'] = fine_tuned_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
fine_tuned_prediction = fine_tuned_prediction[['subject', 'setting', 'model', 'signal']]

fine_tuned_prediction.head()

In [ ]:
r_ppg_path = utils.join_paths(signals_dir, 'r_ppg_predictions.csv')

r_ppg_prediction = pd.read_csv(r_ppg_path)
r_ppg_prediction['signal'] = r_ppg_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
r_ppg_prediction = r_ppg_prediction[['subject', 'setting', 'model', 'signal']]

r_ppg_prediction.head()

In [ ]:
transformer_path = utils.join_paths(signals_dir, 'transformer_predictions.csv')

transformer_prediction = pd.read_csv(transformer_path)
transformer_prediction['signal'] = transformer_prediction['signal'].apply(eval).apply(np.array)

# Only keep the columns that are needed
transformer_prediction = transformer_prediction[['subject', 'setting', 'model', 'signal']]

transformer_prediction.head()

In [ ]:
predictions = pd.concat([
    raft_predictions,
    flownet_predictions,
    pretrained_predictions,
    unsupervised_predictions,
    fine_tuned_prediction,
    r_ppg_prediction,
    transformer_prediction,
])
len(predictions)

## Step 2: Extract the frequencies using a sliding window approach

In [ ]:
from respiration.dataset import VitalCamSet

dataset = VitalCamSet()

In [ ]:
import itertools
from tqdm.auto import tqdm
import respiration.analysis as analysis

# rPPG lowpass and highpass frequencies
# lowpass = 0.7
# highpass = 2.5

# Breathing lowpass and highpass frequencies
lowpass = 0.08
highpass = 0.5

subjects = predictions['subject'].unique()
settings = predictions['setting'].unique()

frequency_predictions = []

for (subject, setting) in tqdm(itertools.product(subjects, settings), total=len(subjects) * len(settings)):
    scenario_predictions = predictions[
        (predictions['subject'] == subject) &
        (predictions['setting'] == setting)]
    
    if len(scenario_predictions) == 0:
        continue

    # Get the ground truth signal
    gt_signal = dataset.get_breathing_signal(subject, setting)

    # Remove the first entry of the ground truth signal, because the prediction is run on a time shifted input
    gt_signal = gt_signal[1:]

    # Get the ground truth frequencies over a sliding window
    gt_frequencies = analysis.sliding_window_analysis(gt_signal, 30, lowpass, highpass)

    for model in scenario_predictions['model'].unique():
        model_scenario_predictions = scenario_predictions[scenario_predictions['model'] == model]
        predicted_signal = model_scenario_predictions['signal'].values[0]
        pred_frequencies = analysis.sliding_window_analysis(predicted_signal, 30, lowpass, highpass)

        for metric in gt_frequencies:
            frequency_predictions.append({
                'subject': subject,
                'setting': setting,
                'model': model,
                'metric': metric,
                'gt_frequencies': gt_frequencies[metric],
                'pred_frequencies': pred_frequencies[metric]
            })

In [ ]:
frequencies_df = pd.DataFrame(frequency_predictions)

## Step 3: Evaluate the performance of the models

In [ ]:
import scipy.stats as stats

models = frequencies_df['model'].unique()
metrics = frequencies_df['metric'].unique()

evaluation_results = []

for (model, metric) in itertools.product(models, metrics):
    model_metric_df = frequencies_df[
        (frequencies_df['model'] == model) &
        (frequencies_df['metric'] == metric)]

    # Concatenate the predicted and ground truth frequencies
    gt_frequencies = np.concatenate(model_metric_df['gt_frequencies'].values)
    pred_frequencies = np.concatenate(model_metric_df['pred_frequencies'].values)
    
    gt_frequencies = gt_frequencies[:len(pred_frequencies)]

    mae = np.mean(np.abs(gt_frequencies - pred_frequencies))
    rmse = np.sqrt(np.mean((gt_frequencies - pred_frequencies) ** 2))
    
    # Multiply the frequencies by 60 to get the bpm
    mae *= 60
    rmse *= 60
    
    corr, p = stats.pearsonr(gt_frequencies.flatten(), pred_frequencies.flatten())

    evaluation_results.append({
        'model': model,
        'metric': metric,
        'mae': mae,
        'rmse': rmse,
        'corr': corr,
        'p': p
    })

In [ ]:
evaluation_df = pd.DataFrame(evaluation_results)
evaluation_df

In [ ]:
analysis_dir = utils.dir_path('outputs', 'analysis')
evaluation_path = utils.join_paths(analysis_dir, 'sliding_window_evaluation.csv')
evaluation_df.to_csv(evaluation_path, index=False)

In [ ]:
evaluation_df[evaluation_df['metric'] == 'psd']

## Step 4: Visualize the results

In [ ]:
figure_dir = utils.dir_path('outputs', 'figures')

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import respiration.utils as utils

metric = 'psd'

fig, axs = plt.subplots(5, 5, figsize=(20, 15))

# Add some space between the plots
fig.tight_layout(pad=5.0)

for idx, model in enumerate(models):
    model_metric_df = frequencies_df[
        (frequencies_df['model'] == model) &
        (frequencies_df['metric'] == metric)]

    # Concatenate the predicted and ground truth frequencies
    ground_truth = np.concatenate(model_metric_df['gt_frequencies'].values)
    prediction = np.concatenate(model_metric_df['pred_frequencies'].values)

    ax = axs[idx // 5, idx % 5]

    ax.scatter(prediction, ground_truth, alpha=0.5)

    # Add a regression line
    x = prediction
    y = ground_truth
    m, b = np.polyfit(x, y, 1)
    ax.plot(x, m * x + b, color='red')

    ax.set_xlabel('Prediction')
    ax.set_ylabel('Ground truth')
    ax.set_title(model)

utils.savefig(fig, figure_dir, f'model_correlation_{metric}')